In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain_pinecone import PineconeVectorStore


In [2]:
#Extract data from PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [3]:
extracted_data = load_pdf("data/")

In [4]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [5]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [6]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    
    return embeddings

In [7]:
embeddings = download_hugging_face_embeddings()

In [8]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [9]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [10]:
PINECONE_API_KEY = '9a068d8b-d1c4-458a-97c4-969589ebe663'
index_name="medical-chatbot"

In [11]:
from pinecone import Pinecone
pc = Pinecone(api_key = PINECONE_API_KEY)
pc.list_indexes().names() # to check if my index exsist

index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [12]:
#Creating Embeddings for Each of The Text Chunks & storing
docs_chunks =[t.page_content for t in text_chunks]

vectorstore = PineconeVectorStore(
index=index,
pinecone_api_key = PINECONE_API_KEY,
embedding=embeddings,
namespace="medicalChatBot",
index_name='medical-chatbot'
)

vectorstore.add_texts(texts=[t.page_content for t in text_chunks])

['fe664ab5-6ee4-43ca-9465-3c6f7e85f3ca',
 'f6a4afa6-3757-424f-ab81-20b5719fe4e2',
 'e2f2636d-5a01-4b8a-bda8-d6db0c40d232',
 '99ee1103-6e60-462b-83c6-d5e0a5c5a2b9',
 'e1b2b301-8f90-417b-be35-3f6566ba762e',
 '15a2d6af-6ef4-4b92-b867-336399dfa5ae',
 'fd507bc4-f444-47d6-b305-e65afcab4587',
 '7be61b50-9168-4988-afce-4d32a7714428',
 '20761445-6053-418a-9ab4-be3f005119ee',
 'f01f2576-e7a6-478b-a37d-ae4554740219',
 'cb059a1b-0d37-4e01-836c-4d4e652d7c57',
 '978ceb96-448b-4a62-a8da-b0354496a866',
 '62e59d95-9674-4b50-8b58-6a8a1f00c791',
 '34ec7932-1bc2-4bcd-88e1-f8c52035d87e',
 'b6e80662-9dc1-4c1a-8780-7cce016cf153',
 '5a9b8eb9-4f3c-496b-8e36-820f87ebcdf5',
 'e6a6d7a7-7e93-448b-aa09-bce5957fa785',
 '2f8da8fe-3e58-42ae-a1d1-420c3d49108d',
 'ed31c4ce-2693-4689-9955-1fd5ed7d6b79',
 '75452c07-0d34-4435-9bb1-942fad32d077',
 '379dead1-6f27-4ecf-a0b6-0d8402be809f',
 'ff963606-2904-4966-8c49-14fae1264f08',
 'b8535128-70de-4a92-aacc-d554edc216ed',
 '2ceb25a7-1046-436e-a4e4-fe843c646f3f',
 '87b051c5-9d5e-

In [13]:
query = "What is Acne"

docs=vectorstore.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='Acidosis seeRespiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when thepores of the skin become clogged with oil, dead skincells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is'), Document(page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25Acne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceousglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'), Document(page_content='ent purposes. For example, lotions, soaps, gels, andcreams containing benzoyl peroxide or tretinoin may beused to clear up mild to moderately severe acne.Isotretinoin (Accutane) is prescribed only for verysevere, disfiguring acne.\nAcne is a skin condition that